In [24]:
import calendar
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [25]:
df = pd.read_csv('C:/Users/Dear Gabriel/Desktop/Motor_Vehicle_Crashes_3_Year_Window.csv')
df.head()

,Year,Crash Descriptor,Time,Date,Day of Week,Police Report,Lighting Conditions,Municipality,Collision Type Descriptor,County Name,Road Descriptor,Weather Conditions,Traffic Control Device,Road Surface Conditions,DOT Reference Marker Location,Pedestrian Bicyclist Action,Event Descriptor,Number of Vehicles Involved
0,2016,Property Damage Accident,17:14,12/31/2016,Saturday,Y,Dark-Road Unlighted,MARTINSBURG,OTHER,LEWIS,Curve and Grade,Cloudy,None,Wet,26 74061244,Not Applicable,Deer,1
1,2016,Property Damage Accident,5:30,12/31/2016,Saturday,Y,Dark-Road Unlighted,BROOKHAVEN,SIDESWIPE,SUFFOLK,Straight and Level,Clear,None,Dry,NaN,Not Applicable,"Other Motor Vehicle, Collision With",2
2,2016,Property Damage & Injury Accident,11:02,12/31/2016,Saturday,Y,Daylight,MILFORD,LEFT TURN (3),OTSEGO,Straight and Level,Cloudy,No Passing Zone,Wet,7 94033029,Not Applicable,"Other Motor Vehicle, Collision With",2
3,2016,Injury Accident,15:20,12/31/2016,Saturday,N,Unknown,KINGS,Unknown,KINGS,Unknown,Unknown,Unknown,Unknown,NaN,Not Applicable,"Other Motor Vehicle, Collision With",2
4,2016,Property Damage Accident,8:52,12/31/2016,Saturday,Y,Daylight,RENSSELAER,LEFT TURN (3),RENSSELAER,Straight and Grade,Clear,Traffic Signal,Dry,NaN,Not Applicable,"Other Motor Vehicle, Collision With",2


In [26]:
list(df.columns)

['Year',
 'Crash Descriptor',
 'Time',
 'Date',
 'Day of Week',
 'Police Report',
 'Lighting Conditions',
 'Municipality',
 'Collision Type Descriptor',
 'County Name',
 'Road Descriptor',
 'Weather Conditions',
 'Traffic Control Device',
 'Road Surface Conditions',
 'DOT Reference Marker Location',
 'Pedestrian Bicyclist Action',
 'Event Descriptor',
 'Number of Vehicles Involved']

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895916 entries, 0 to 895915
Data columns (total 18 columns):
Year                             895916 non-null int64
Crash Descriptor                 895916 non-null object
Time                             895916 non-null object
Date                             895916 non-null object
Day of Week                      895916 non-null object
Police Report                    895916 non-null object
Lighting Conditions              895916 non-null object
Municipality                     883939 non-null object
Collision Type Descriptor        895916 non-null object
County Name                      895916 non-null object
Road Descriptor                  895916 non-null object
Weather Conditions               895916 non-null object
Traffic Control Device           895916 non-null object
Road Surface Conditions          895916 non-null object
DOT Reference Marker Location    313601 non-null object
Pedestrian Bicyclist Action      895916 non-null o

 All the columns have 895916 entries except DOT Reference Marker Location. Ths is telling us  that There are null values found in the data. We can double check it.

In [28]:
df.isnull().sum().any()

True

There is only one column with a null value: " DOT Reference Marker Location ". We can remove it. Therefore we won't analyse the accident by using DOT Reference Marker Location . In addition, we are goint to remove the columns that we won't need for futher analyse.

In [29]:
df.drop(['Police Report','Lighting Conditions','Municipality','Collision Type Descriptor','Road Descriptor','Weather Conditions','Traffic Control Device','Road Surface Conditions','DOT Reference Marker Location','Pedestrian Bicyclist Action','Event Descriptor'],axis=1 ,inplace=True)

In [30]:
df.columns

Index(['Year', 'Crash Descriptor', 'Time', 'Date', 'Day of Week',
       'County Name', 'Number of Vehicles Involved'],
      dtype='object')

In [31]:
df.rename(columns=lambda x:x.replace(' ','_').lower(), inplace=True)

new columns names

In [32]:
df.columns

Index(['year', 'crash_descriptor', 'time', 'date', 'day_of_week',
       'county_name', 'number_of_vehicles_involved'],
      dtype='object')

In [33]:
df.head()

,year,crash_descriptor,time,date,day_of_week,county_name,number_of_vehicles_involved
0,2016,Property Damage Accident,17:14,12/31/2016,Saturday,LEWIS,1
1,2016,Property Damage Accident,5:30,12/31/2016,Saturday,SUFFOLK,2
2,2016,Property Damage & Injury Accident,11:02,12/31/2016,Saturday,OTSEGO,2
3,2016,Injury Accident,15:20,12/31/2016,Saturday,KINGS,2
4,2016,Property Damage Accident,8:52,12/31/2016,Saturday,RENSSELAER,2


In [34]:
df['month'] = pd.DatetimeIndex(df['date']).month # new colum showing months
df['day'] = pd.DatetimeIndex(df['date']).day # new colum showing days
df['hour'] = pd.DatetimeIndex(df['time']).hour # new colum showing hours unit


In [38]:
df.head()

,year,crash_descriptor,time,date,day_of_week,county_name,number_of_vehicles_involved,month,day,hour
0,2016,Property Damage Accident,17:14,12/31/2016,Saturday,LEWIS,1,12,31,17
1,2016,Property Damage Accident,5:30,12/31/2016,Saturday,SUFFOLK,2,12,31,5
2,2016,Property Damage & Injury Accident,11:02,12/31/2016,Saturday,OTSEGO,2,12,31,11
3,2016,Injury Accident,15:20,12/31/2016,Saturday,KINGS,2,12,31,15
4,2016,Property Damage Accident,8:52,12/31/2016,Saturday,RENSSELAER,2,12,31,8


In [39]:
list(df.month.unique())
#list(df.day.unique())

[12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [40]:
def _time_of_day(hour):
    
    if (hour > 4) & (hour <= 11):
        tod = "morning"
    elif (hour > 11) & (hour <= 16):
        tod = "afternoon"
    elif (hour > 16) & (hour <= 19):
        tod = "evening"
    else:
        tod = "night"
    
    return tod

In [41]:
df["time_of_day"] = df["hour"].apply(lambda row: _time_of_day(row))

In [42]:
df.head()

,year,crash_descriptor,time,date,day_of_week,county_name,number_of_vehicles_involved,month,day,hour,time_of_day
0,2016,Property Damage Accident,17:14,12/31/2016,Saturday,LEWIS,1,12,31,17,evening
1,2016,Property Damage Accident,5:30,12/31/2016,Saturday,SUFFOLK,2,12,31,5,morning
2,2016,Property Damage & Injury Accident,11:02,12/31/2016,Saturday,OTSEGO,2,12,31,11,morning
3,2016,Injury Accident,15:20,12/31/2016,Saturday,KINGS,2,12,31,15,afternoon
4,2016,Property Damage Accident,8:52,12/31/2016,Saturday,RENSSELAER,2,12,31,8,morning


In [43]:
df['date']=pd.to_datetime(df[['month','day','year', 'day_of_week', 'time_of_day']])

ValueError: extra keys have been passed to the datetime assemblage: [day_of_week,time_of_day]

In [17]:
# Drop columns hour, day, month, year, weekday

df.drop(['time','day','hour'], axis=1, inplace=True)



# Dataframe columns

list(df.columns)

['year',
 'crash_descriptor',
 'date',
 'day_of_week',
 'county_name',
 'number_of_vehicles_involved',
 'month',
 'time_of_day']

The final cleaning step consists of removing duplicated entries(if there are) from the data frame.

In [18]:
df.duplicated().sum()



# Print duplicated rows.



452633

In [19]:
df[df.duplicated()]

,year,crash_descriptor,date,day_of_week,county_name,number_of_vehicles_involved,month,time_of_day
14,2016,Injury Accident,12/31/2016,Saturday,WESTCHESTER,2,12,afternoon
28,2016,Injury Accident,12/31/2016,Saturday,WESTCHESTER,2,12,afternoon
37,2016,Property Damage Accident,12/31/2016,Saturday,SUFFOLK,2,12,afternoon
44,2016,Property Damage Accident,12/31/2016,Saturday,SUFFOLK,2,12,morning
59,2016,Property Damage Accident,12/31/2016,Saturday,WESTCHESTER,2,12,evening
...,...,...,...,...,...,...,...,...
895906,2014,Property Damage Accident,1/1/2014,Wednesday,OTSEGO,1,1,night
895907,2014,Property Damage Accident,1/1/2014,Wednesday,SUFFOLK,2,1,morning
895909,2014,Property Damage Accident,1/1/2014,Wednesday,KINGS,2,1,morning
895910,2014,Property Damage Accident,1/1/2014,Wednesday,ERIE,1,1,morning


In [20]:
# Shape of the dataframe before removing duplicated columns

df.shape





# Drop duplicates.

df.drop_duplicates(inplace=True)



# Shape after dropping duplicates.

df.shape



(443283, 8)

In [21]:
df.head()

,year,crash_descriptor,date,day_of_week,county_name,number_of_vehicles_involved,month,time_of_day
0,2016,Property Damage Accident,12/31/2016,Saturday,LEWIS,1,12,evening
1,2016,Property Damage Accident,12/31/2016,Saturday,SUFFOLK,2,12,morning
2,2016,Property Damage & Injury Accident,12/31/2016,Saturday,OTSEGO,2,12,morning
3,2016,Injury Accident,12/31/2016,Saturday,KINGS,2,12,afternoon
4,2016,Property Damage Accident,12/31/2016,Saturday,RENSSELAER,2,12,morning


In [22]:


accidents_month = df.groupby(df['date'].dt.month).count().date

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:

accidents_month.index=[calendar.month_name[x] for x in range(1,13)]

In [ ]:
accidents_month

In [ ]:
df['date'] = pd.to_datetime(df['date'])
start = pd.Timestamp(2015,1,1)
end = pd.Timestamp(2016,1,1)
df[(df['date']>=start) & (df['date'] < end)]

In [ ]:
% matplotlib inline

plt.style.use('ggplot')



# plot accidents per month

accidents_month.plot(kind='bar',figsize=(12,7), color='red', alpha=0.5)



# title and x,y labels

plt.title('Accidents In Las Vegas Between 2014 And 2016', fontsize=20)

plt.xlabel('Month',fontsize=16)

plt.ylabel('Number of accidents',fontsize=16);

In [ ]:
# Number of accident per day of the week

accidents_day = df.groupby(df['date'].dt.dayofweek).count().date



# Replace the day integers by day names.

accidents_day.index=[calendar.day_name[x] for x in range(0,7)]



# plot accidents per day

accidents_day.plot(kind='bar',figsize=(12,7), color='indigo', alpha=0.5)



# title and x,y labels

plt.title('Accidents In Las Vegas Between 2014 And 2016', fontsize=20)

plt.xlabel('Day of the week',fontsize=16)

plt.ylabel('Number of accidents',fontsize=16);

In [ ]:
accidents = df.groupby(df['date'].dt.date).count().date



accidents.plot(figsize=(13,8), color='purple')



# sunday accidents

sundays = df.groupby(df[df['date'].dt.dayofweek==6].date.dt.date).count().date

plt.scatter(sundays.index, sundays, color='green', label='sunday')



# friday accidents

friday = df.groupby(df[df['date'].dt.dayofweek==4].date.dt.date).count().date

plt.scatter(friday.index, friday, color='yellow', label='friday')



# Title, x label and y label

plt.title('Accidents In Las Vegas Between 2014 and 2016', fontsize=20)

plt.xlabel('Date',fontsize=16)

plt.ylabel('Number of accidents per day',fontsize=16);

plt.legend()

In [ ]:
# Number of accident per hour

accidents_timeOfday = df.groupby(df['date'].dt.time_of_day).count().date



# plot accidents per hour

accidents_hour.plot(kind='bar',figsize=(12,7), color='gold', alpha=0.5)



# title and x,y labels

plt.title('Accidents In Las Vegas Between 2014 And 2016', fontsize=20)

plt.xlabel('Time Of The Day',fontsize=16)

plt.ylabel('Number of accidents',fontsize=16);